# Set puzzle parameters and create AoC Session

In [ ]:
# set puzzle parameters
PUZZLE_DAY = 10
PUZZLE_YEAR = 2024

# import from local packages
from aoc_solver import AoCSession, AoCSolver, AoCTester
AoC_SESSION = AoCSession.from_file()

# Import additional packages

In [ ]:
# import from standard library packages
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Self, Tuple

# import from third-party packages
from polars import DataFrame
from tqdm.notebook import tqdm

# import from local packages
from aoc_solver import AoCMap, Direction

# Create solver class and instance

In [ ]:
@dataclass
class TrailNode:
    height: int
    position: Tuple[int, int]
    ascents: Dict[Direction, Optional[Self]] = field(default_factory=dict)

@dataclass
class TrailHead:
    head: TrailNode
    tails: List[TrailNode] = field(default_factory=list)
    cardinal_directions: List[Direction] = field(default_factory=list)
        
    def __post_init__(self):
        self.cardinal_directions = [Direction.N, Direction.E, Direction.S, Direction.W]
        
    def map_trails(self, data: AoCMap) -> None:
        
        def explore_from_node(
            data: AoCMap,
            node: TrailNode,
            directions = self.cardinal_directions,
            tails = self.tails,
        ) -> None:
            for direction in directions:
                data.position = node.position
                data.walk(steps=1, direction=direction)
                new_position = data.position
                new_height = data.look(0)
                if new_position == node.position or new_height != node.height + 1:
                    continue
                new_node = TrailNode(height=new_height, position=new_position)
                if new_height == 9:
                    tails.append(new_node)
                    continue
                node.ascents[direction] = new_node
                explore_from_node(data=data, node = new_node)
        
        explore_from_node(data=data, node=self.head)
        
    @property
    def score(self) -> int:
        return len({tail.position for tail in self.tails})
    
    @property
    def rating(self) -> int:
        return len([tail.position for tail in self.tails])

class Solver(AoCSolver):
    
    @staticmethod
    def build_trails(data: DataFrame) -> List[TrailHead]:
        topo_map = AoCMap(data)
        start_positions = topo_map.element_positions[0]
        trailheads = [
            TrailHead(TrailNode(height=0, position=position))
            for position in start_positions
        ]
        for trailhead in tqdm(trailheads): 
            trailhead.map_trails(data=topo_map)
        return trailheads
    
    def solve_part1(self, data: DataFrame) -> int:
        trailheads = self.build_trails(data)
        return sum(trailhead.score for trailhead in trailheads)

    def solve_part2(self, data: DataFrame) -> int:
        trailheads = self.build_trails(data)
        return sum(trailhead.rating for trailhead in trailheads)

In [ ]:
solver = Solver(PUZZLE_YEAR, PUZZLE_DAY, AoC_SESSION)

# Build part 1 test case(s)

In [ ]:
puzzle_instructions = solver.puzzle_instructions
puzzle_input = solver.puzzle_input.create_polars(dtype=int ,separator='')

part1_test_input = solver.get_value_after('Here\'s a larger example:').create_polars(dtype=int ,separator = '')
print(f'{part1_test_input=}\n')

part1_test_output = solver.get_value_after('all trailheads is ').as_int
print(f'{part1_test_output=}\n')

In [ ]:
part_1_tester = AoCTester()
part_1_tester.add_test_case(part1_test_input, part1_test_output)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Determine part 1 solution

In [ ]:
%%time
part1_solution = solver.solve_part1(puzzle_input)
print(f'{part1_solution=}\n')

# Add part 1 solution to part 1 test cases

In [ ]:
part_1_tester.add_test_case(puzzle_input, part1_solution)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Build part 2 test case(s)

In [ ]:
solver.download_instructions(overwrite=True)

In [ ]:
part2_test_input = part1_test_input
print(f'{part2_test_input=}\n')

part2_test_output = solver.get_value_after('example topographic map is ').as_int
print(f'{part2_test_output=}\n')

In [ ]:
part_2_tester = AoCTester()
part_2_tester.add_test_case(part2_test_input, part2_test_output)

In [ ]:
%%time
part_2_tester.run_tests(solver.solve_part2)

# Determine part 2 solution

In [ ]:
%%time
part2_solution = solver.solve_part2(puzzle_input)
print(f'{part2_solution=}\n')